In [62]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, make_pipeline

In [12]:
df = pd.read_csv('train.csv')

In [13]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

In [15]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [17]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
298,1,male,NaN,0,0,30.5000,S
884,3,male,25.00,0,0,7.0500,S
247,2,female,24.00,0,2,14.5000,S
478,3,male,22.00,0,0,7.5208,S
305,1,male,0.92,1,2,151.5500,S


In [31]:
# imputation transformer
trf1 = ColumnTransformer(
    transformers=[
        ('age_imputed', SimpleImputer(),[2]), 
        ('embarked_imputed', SimpleImputer(strategy='most_frequent'), [6])
    ], 
    remainder = 'passthrough'
)

In [46]:
# One Hot encoding transformer 
trf2 = ColumnTransformer(
    transformers=[
        ('sex_ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1]),
        ('embarked_ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [6]),
    ], 
    remainder = 'passthrough'
)

In [48]:
# Scaling transformer 
trf3 = ColumnTransformer(
    transformers=[
       ('scaled', MinMaxScaler(), slice(0,10)) 
    ], 
)

In [49]:
# Feature Selection 
trf4 = SelectKBest(score_func=chi2, k=8)

In [50]:
# Model Building 
trf5 = DecisionTreeClassifier()

In [39]:
# Creating Pipeline

In [51]:
pipe = Pipeline([
     ('trf1', trf1),
     ('trf2', trf2), 
     ('trf3', trf3), 
     ('trf4', trf4), 
     ('trf5', trf5),
   ]
)

In [52]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputed',
                                                  SimpleImputer(), [2]),
                                                 ('embared_imputed',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [-1])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1]),
                                                 ('embarked_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scaled', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000012934DD2020>)),
                ('trf5', DecisionTreeClassifier())])

In [54]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('age_imputed', SimpleImputer(), [2]),
                                 ('embared_imputed',
                                  SimpleImputer(strategy='most_frequent'),
                                  [-1])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('sex_ohe',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1]),
                                 ('embarked_ohe',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [6])]),
 'trf3': ColumnTransformer(transformers=[('scaled', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000012934DD2020>),
 'trf5': DecisionTreeC

In [56]:
pipe.feature_names_in_

array(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'],
      dtype=object)

In [58]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('age_imputed', SimpleImputer(), [2]),
                                ('embared_imputed',
                                 SimpleImputer(strategy='most_frequent'),
                                 [-1])])

In [61]:
pipe.named_steps['trf1']['age_imputed']

SimpleImputer()

In [66]:
y_predict = pipe.predict(X_test)

In [68]:
accuracy = accuracy_score(y_test, y_predict)

In [69]:
accuracy

0.6233183856502242